In [1]:
import torch
import torch.optim as optim
from tqdm import tqdm
from torchtext.vocab import FastText
from utils.preprocess import MINDIterator
from utils.utils import getVocab,getLoss,getLabel,constructBasicDict,run_eval,run_train
from models.NPA import NPAModel

## setting up the *NPA* model

### define paths and hyperparameters, load data

all these hyper parameters are fixed according to the paper [\[23\] Npa Neural news recommendation with personalized attention](https://dl.acm.org/doi/abs/10.1145/3292500.3330665)

- *mode*: data to read (*small*/*large*)

- *batch_size*: size of each minibatch

- *title_size*: max word capacity of title

- *his_size*: max record capacity of click history

- *npratio*: number of negtive sampling

- *dropout_p*: probability of dropout layer

- *filter_num*: number of kernels in 1D CNN, which is also embedding dimension of news/user

- *embedding_dim*: word embedding dimension

- *user_dim*: user id embedding dimension

- *preference_dim*: user preference embedding dimension

In [2]:
hparams = {
    'mode':'small',
    'batch_size':5,#100,
    'title_size':30,
    'his_size':50,   
    'npratio':4,     
    'dropout_p':0.2,
    'filter_num':400,
    'embedding_dim':300,
    'user_dim':50,
    'preference_dim':200,
    'metrics':'group_auc,ndcg@4,mean_mrr',
    'gpu':'cuda:0'
}

# customize your path here
news_file = r'D:\Data\NR_data\dev\news.tsv'
behavior_file_train = r'D:\Data\NR_data\dev\behaviors_train.tsv'
behavior_file_test = r'D:\Data\NR_data\dev\behaviors_test.tsv'
save_path = 'models/model_param/NPA.model'

# if user2id,word2id,news2id hasn't been constructed
if not os.path.exists('data/vocab_'+hparams['mode']+'.pkl'):
    constructBasicDict(news_file,behavior_file_train,hparams['mode'],['title','category','subcategory'])

device = torch.device(hparams['gpu']) if torch.cuda.is_available() else torch.device("cpu")

iterator_train = MINDIterator(hparams=hparams,news_file=news_file,behaviors_file=behavior_file_train)

# torchtext.Vocab.vocab object
vocab = iterator_train.vocab
embedding = FastText('simple',cache='.vector_cache')
vocab.load_vectors(embedding)

In [3]:
# you can load my model or train yours
if os.path.exists(save_path):
    npaModel = NPAModel(vocab=vocab,hparams=hparams)
    npaModel.load_state_dict(torch.load(save_path))
    npaModel.to(device).eval()

else:
    npaModel = NPAModel(vocab=vocab,hparams=hparams).to(device)
    npaModel.train()

### train model

In [4]:
if npaModel.training:
    loss_func = getLoss(npaModel)
    optimizer = optim.Adam(npaModel.parameters(),lr=0.001)

    npaModel = run_train(npaModel,iterator_train,optimizer,loss_func)

epoch 0 , step 100 , total_loss: 1.6368, batch_loss: 1.6072: : 143it [00:05, 25.29it/s]
epoch 1 , step 100 , total_loss: 1.6101, batch_loss: 1.6067: : 143it [00:03, 37.54it/s]
epoch 2 , step 100 , total_loss: 1.6086, batch_loss: 1.6092: : 143it [00:03, 39.45it/s]
epoch 3 , step 100 , total_loss: 1.6075, batch_loss: 1.6172: : 143it [00:03, 40.54it/s]
epoch 4 , step 100 , total_loss: 1.6079, batch_loss: 1.6036: : 143it [00:03, 40.74it/s]
epoch 5 , step 100 , total_loss: 1.6101, batch_loss: 1.6083: : 143it [00:03, 36.96it/s]
epoch 6 , step 100 , total_loss: 1.6079, batch_loss: 1.6073: : 143it [00:03, 38.27it/s]
epoch 7 , step 100 , total_loss: 1.6102, batch_loss: 1.6156: : 143it [00:03, 36.08it/s]
epoch 8 , step 100 , total_loss: 1.6105, batch_loss: 1.6133: : 143it [00:03, 37.09it/s]
epoch 9 , step 100 , total_loss: 1.6093, batch_loss: 1.6175: : 143it [00:03, 38.13it/s]


### test & evaluate

In [5]:
iterator_test = MINDIterator(hparams,news_file,behavior_file_test)
iterator_test.npratio = -1
test = iterator_test.load_data_from_file()

npaModel.eval()
npaModel.npratio = -1
run_eval(npaModel,test)

{'group_auc': 0.5004, 'ndcg@4': 0.168, 'mean_mrr': 0.1901}

### save the model

In [6]:
npaModel.npratio = 4
torch.save(npaModel.state_dict(), save_path)